In [ ]:
# IMDB Datasets

In [ ]:
import tensorflow_dataset as tfds
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
form tensorflow.keras import models, layers
     

In [ ]:
datasets, info = tfds.load(    # unonomus variable
            "imdb_review/plain_text",
            split=["train", "test[:50%]",  "test[50%:]"],
            as_supervised=True, # not activeated, the data will be as dict, now as tuple
            with_info =True    #
)

dataset_train_original = datasets[0] 
dataset_validate_original =  datasets[1] 
dataset_test_original = datasets[2] 
info

In [ ]:
# Look at your beeping data